In [2]:
!pip install pymysql

In [2]:
import pymysql


In [7]:
conn = pymysql.connect(
        host= "localhost",
        database="sagemcom-text2sql",
        user="root",
        password="Cesenacesena1!",
        port=3306
    )
cur = conn.cursor()

In [7]:
conn = pymysql.connect(
        host= "sagemcom-text2sql-db.ceginbcdboy1.eu-central-1.rds.amazonaws.com",
        database="sagemcom-text2sql-mysqldb",#sagemcom-text2sql-db
        user="admin",
        password="testsagemcomtext2sql",
        port=3306
    )
cur = conn.cursor()

In [9]:
cur.execute("""SELECT 
    kcu.table_name AS referencing_table,
    kcu.column_name AS referencing_column,
    kcu.referenced_table_name AS referenced_table,
    kcu.referenced_column_name AS referenced_column
FROM 
    information_schema.key_column_usage AS kcu
JOIN 
    information_schema.referential_constraints AS rc
ON 
    kcu.constraint_name = rc.constraint_name
    AND kcu.table_schema = rc.constraint_schema
WHERE 
    kcu.table_schema = 'sagemcom-text2sql-mysqldb';""")

24

In [34]:
results = cur.fetchall()
for row in results:
   print(row)

('customer_dim', 'Geography_ID', 'geography_dim', 'Geography_ID')
('customer_dim', 'Age_Group_ID', 'age_group_dim', 'Age_Group_ID')
('date_dim', 'Month_ID', 'month_dim', 'Month_ID')
('date_dim', 'Year_ID', 'year_dim', 'Year_ID')
('date_dim', 'Time_ID', 'time_dim', 'Time_ID')
('product_dim', 'Subcategory_ID', 'subcategory_dim', 'Subcategory_ID')
('product_dim', 'Manufacturer_ID', 'manufacturer_dim', 'Manufacturer_ID')
('promotion_dim', 'Start_Date_ID', 'date_dim', 'Date_ID')
('promotion_dim', 'End_Date_ID', 'date_dim', 'Date_ID')
('return_fact', 'Sales_ID', 'sales_fact', 'Sales_ID')
('return_fact', 'Date_ID', 'date_dim', 'Date_ID')
('return_fact', 'Product_ID', 'product_dim', 'Product_ID')
('return_fact', 'Customer_ID', 'customer_dim', 'Customer_Surrogate_Key')
('return_fact', 'Store_ID', 'store_dim', 'Store_ID')
('sales_fact', 'Date_ID', 'date_dim', 'Date_ID')
('sales_fact', 'Customer_ID', 'customer_dim', 'Customer_Surrogate_Key')
('sales_fact', 'Product_ID', 'product_dim', 'Product_ID

In [41]:
database="sagemcom-text2sql"

# Query 1: Get all table and column details
schema_query = f"""
SELECT c.table_name, c.column_name, c.data_type, c.is_nullable, c.column_default
FROM information_schema.columns c
WHERE c.table_schema = '{database}'
ORDER BY c.table_name, c.ordinal_position;
"""

# Query 2: Get primary keys for each table
primary_keys_query = f"""
SELECT kcu.table_name, kcu.column_name
FROM information_schema.table_constraints tc
JOIN information_schema.key_column_usage kcu
ON tc.constraint_name = kcu.constraint_name
AND tc.table_schema = kcu.table_schema
WHERE tc.constraint_type = 'PRIMARY KEY' AND tc.table_schema = '{database}';
"""

# Query 3: Get foreign keys and references for each table
foreign_keys_query = f"""
SELECT 
    kcu.table_name AS referencing_table,
    kcu.column_name AS referencing_column,
    kcu.referenced_table_name AS referenced_table
FROM 
    information_schema.key_column_usage AS kcu
JOIN 
    information_schema.referential_constraints AS rc
ON 
    kcu.constraint_name = rc.constraint_name
    AND kcu.table_schema = rc.constraint_schema
WHERE 
    kcu.table_schema = '{database}';
"""

# Execute the queries and fetch results
cur.execute(schema_query)
columns_details = cur.fetchall()

cur.execute(primary_keys_query)
primary_keys = cur.fetchall()

cur.execute(foreign_keys_query)
foreign_keys = cur.fetchall()

# Organize details into dictionaries
schema_dict = {}
primary_keys_dict = {table: set() for table, _ in primary_keys}
foreign_keys_dict = {table: {} for table, _, _ in foreign_keys}

for table, column in primary_keys:
    primary_keys_dict[table].add(column)

for table, column, ref_table in foreign_keys:
    foreign_keys_dict[table][column] = ref_table

# Build the schema dictionary
for table, column, data_type, is_nullable, column_default in columns_details:
    if table not in schema_dict:
        schema_dict[table] = []
    column_desc = f"{column} ({data_type.upper()}"

    # Add primary key annotation
    if column in primary_keys_dict.get(table, []):
        column_desc += ", PRIMARY KEY"

    # Add foreign key reference
    if column in foreign_keys_dict.get(table, {}):
        column_desc += f", FOREIGN KEY REFERENCES {foreign_keys_dict[table][column]}"

    column_desc += ")"
    schema_dict[table].append(column_desc)

# Format the schema string
schema_str = ""
for table, columns in schema_dict.items():
    schema_str += f"{table}:\n"
    for col in columns:
        schema_str += f"- {col}\n"
    schema_str += "\n"

In [43]:
import json
print(json.dumps(schema_dict, indent=4))

{
    "age_group_dim": [
        "Age_Group_ID (INT, PRIMARY KEY)",
        "Age_Range (VARCHAR)"
    ],
    "category_dim": [
        "Category_ID (INT, PRIMARY KEY)",
        "Category_Name (VARCHAR)"
    ],
    "customer_dim": [
        "Customer_ID (INT)",
        "Customer_Surrogate_Key (INT, PRIMARY KEY)",
        "Customer_Name (VARCHAR)",
        "Geography_ID (INT, FOREIGN KEY REFERENCES geography_dim)",
        "Age_Group_ID (INT, FOREIGN KEY REFERENCES age_group_dim)",
        "Start_Date (DATE)",
        "End_Date (DATE)",
        "Is_Current (TINYINT)"
    ],
    "date_dim": [
        "Date_ID (INT, PRIMARY KEY)",
        "Date (DATE)",
        "Day (INT)",
        "Month_ID (INT, FOREIGN KEY REFERENCES month_dim)",
        "Year_ID (INT, FOREIGN KEY REFERENCES year_dim)",
        "Time_ID (INT, FOREIGN KEY REFERENCES time_dim)"
    ],
    "geography_dim": [
        "Geography_ID (INT, PRIMARY KEY)",
        "Country (VARCHAR)",
        "State (VARCHAR)",
        "City (VA

In [1]:
aa = "3306"
int(aa)

3306